In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from datetime import datetime
import os
from glob import glob #para eliminar archivos dentro de carpeta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def inicio_driver(link:str):
    service = Service(ChromeDriverManager().install())
    carpeta_descarga=os.getcwd().replace('src','downloads')
    prefs = {'download.default_directory' : carpeta_descarga,
        "directory_upgrade": True}
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(link)
    driver.maximize_window()
    return driver
def every_downloads_chrome(driver):
    '''Para ver cuando terminan las descargas'''
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl || e.file_url);
        """)

In [3]:
link_indec="https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-2-41"

link_anual_rubro='https://www.indec.gob.ar/ftp/cuadros/economia/indicesexpgr_04.xls'
link_anual_uso='https://www.indec.gob.ar/ftp/cuadros/economia/indicesimpue_04.xls'
link_mensual_rubro='https://www.indec.gob.ar/ftp/cuadros/economia/serie_mensual_indices_expo.xls'
link_mensual_uso='https://www.indec.gob.ar/ftp/cuadros/economia/serie_mensual_indices_impo_ue.xls'

link_anual_indices = 'https://www.indec.gob.ar/ftp/cuadros/economia/expindices_04.xls'

driver = inicio_driver(link_indec)

driver.get(link_anual_rubro)
driver.get(link_anual_uso)
driver.get(link_mensual_rubro)
driver.get(link_mensual_uso)
driver.get(link_anual_indices)



# waits for all the files to be completed and returns the paths
paths = WebDriverWait(driver, 120, 1).until(every_downloads_chrome)

driver.quit()

TimeoutException: Message: 


In [7]:
def lectura_indices_anuales():
    columns_indices_anuales = ["Período","vx","px","qx","d","vm","pm","qm","d2","tot"]
    indices_anuales = pd.read_excel("../downloads/expindices_04.xls",names=columns_indices_anuales, header=5, skipfooter=6)
    indices_anuales.Período = indices_anuales.Período.str.replace("*","", regex=False)
    indices_anuales = (indices_anuales.dropna(axis=1)
        # .drop(["vx","vm","pm","qm","tot"],axis = 1)
        .rename(dict(px = "Índice de precios de las expos", qx = "Índice de cantidad de las expos"), axis=1)
        .set_index("Período"))
    return indices_anuales


In [9]:
indices_anuales = lectura_indices_anuales()

In [12]:
indices_anuales.head(5)

,vx,Índice de precios de las expos,Índice de cantidad de las expos,vm,pm,qm,tot
Período,,,,,,,
1986,19.8,72.7,27.3,21.1,96.4,21.9,75.414938
1987,18.4,76.0,24.2,25.9,94.1,27.5,80.765143
1988,26.4,88.2,29.9,23.7,100.5,23.6,87.761194
1989,27.7,90.1,30.7,18.7,110.3,17.0,81.686310
1990,35.7,87.9,40.7,18.2,113.2,16.0,77.650177


In [15]:
def plot_serie_indices_anuales():
    indices_anuales = lectura_indices_anuales()
    fig = go.Figure()
    # for columna in indices_anuales.columns:
    fig.add_trace(go.Scatter( name = "Índice de precios de las expos", x=indices_anuales.index, y=indices_anuales["Índice de precios de las expos"], mode = "lines+markers"))
    fig.add_trace(go.Scatter( name = "Índice de cantidad de las expos", x=indices_anuales.index, y=indices_anuales["Índice de cantidad de las expos"], mode = "lines+markers"))

    #Propiedades
    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(title_text="índices. Base 2004 = 100")
    fig.update_layout(separators=",.", font_family="Georgia", 
    #  margin ={'t': 0},
    height=600, width=1000,
    xaxis_range = [1985.5,2022.5],
    xaxis = dict(tickmode = "linear"),
    template = 'none',
    title_text="Estancamiento exportador Argentino: un panorama de largo plazo",
    legend = dict(yanchor="top",y=1.1, xanchor="left",x=0.01, orientation = "h")
    )

    #Sequías
    fig.add_annotation(x=1989, y=30.7, text="Sequia", yshift=5, ax=-20,showarrow=True, arrowhead=1)
    fig.add_annotation(x=1996, y=69.3, align="left", text="Sequia", ax=0 , ay=25, yshift=-5, showarrow=True, arrowhead=1)
    fig.add_annotation(x=2009, y=109.3, align="left", text="Sequia y crisis financiera global", ax=0 , ay=30, yshift=-5, showarrow=True, arrowhead=1)
    fig.add_annotation(x=2018, y=111.2, align="left", text="Sequia", ax=0 , ay=30, yshift=-5, showarrow=True, arrowhead=1)
    fig.add_annotation(x=2020, y=108.4, align="left", text="Covid-19", ax=0 , ay=30, yshift=-5, showarrow=True, arrowhead=1)
    #Presidencias
    fig.add_vline(x=1989, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=1999, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2003, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2007, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2011, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2015, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2019, line_width=1, line_dash="dash", line_color="Black")
    fig.add_annotation(text="Alfonsín", x=1987.25, y=140, showarrow=False)
    fig.add_annotation(text="Menemismo", x=1994, y=150, showarrow=False)
    fig.add_annotation(text="Crisis", x=2001, y=160, showarrow=False)
    fig.add_annotation(text="Kircherismo I", x=2005, y=160, showarrow=False)
    fig.add_annotation(text="Kircherismo II", x=2009, y=185, showarrow=False)
    fig.add_annotation(text="Kircherismo III", x=2013, y=160, showarrow=False)
    fig.add_annotation(text="Macrismo", x=2017, y=165, showarrow=False)
    fig.add_annotation(text="Alberto F.", x=2021, y=135, showarrow=False)

    #Nota al pie
    note = 'Fuente: INDEC'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.25,yref='paper', y=-0.15, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return fig

plot_serie_indices_anuales()

    

In [16]:
indices_anuales.columns

Index(['vx', 'Índice de precios de las expos',
       'Índice de cantidad de las expos', 'vm', 'pm', 'qm', 'tot'],
      dtype='object')

In [19]:
def plot_serie_indices_precios_anuales():
    indices_anuales = lectura_indices_anuales()
    fig = go.Figure()
    # for columna in indices_anuales.columns:
    fig.add_trace(go.Scatter( name = "Índice de precios de las expos", x=indices_anuales.index, y=indices_anuales["Índice de precios de las expos"], mode = "lines+markers"))
    fig.add_trace(go.Scatter( name = "Índice de precios de las impos", x=indices_anuales.index, y=indices_anuales["pm"], mode = "lines+markers"))
    fig.add_trace(go.Scatter( name = "Índice de los términos del intercambio", x=indices_anuales.index, y=indices_anuales["tot"], mode = "lines+markers"))
    #Propiedades
    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(title_text="índices. Base 2004 = 100")
    fig.update_layout(separators=",.", font_family="Georgia", 
    #  margin ={'t': 0},
    height=600, width=1000,
    xaxis_range = [1985.5,2022.5],
    xaxis = dict(tickmode = "linear"),
    template = 'none',
    title_text="Los vaivenes del contexto internacional",
    legend = dict(yanchor="top",y=1.1, xanchor="left",x=0.01, orientation = "h")
    )

    #Sequías
    # fig.add_annotation(x=1989, y=30.7, text="Sequia", yshift=5, ax=-20,showarrow=True, arrowhead=1)
    # fig.add_annotation(x=1996, y=69.3, align="left", text="Sequia", ax=0 , ay=25, yshift=-5, showarrow=True, arrowhead=1)
    # fig.add_annotation(x=2009, y=109.3, align="left", text="Sequia y crisis financiera global", ax=0 , ay=30, yshift=-5, showarrow=True, arrowhead=1)
    # fig.add_annotation(x=2018, y=111.2, align="left", text="Sequia", ax=0 , ay=30, yshift=-5, showarrow=True, arrowhead=1)
    # fig.add_annotation(x=2020, y=108.4, align="left", text="Covid-19", ax=0 , ay=30, yshift=-5, showarrow=True, arrowhead=1)
    #Presidencias
    fig.add_vline(x=1989, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=1999, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2003, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2007, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2011, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2015, line_width=1, line_dash="dash", line_color="Black")
    fig.add_vline(x=2019, line_width=1, line_dash="dash", line_color="Black")
    fig.add_annotation(text="Alfonsín", x=1987.25, y=140, showarrow=False)
    fig.add_annotation(text="Menemismo", x=1994, y=150, showarrow=False)
    fig.add_annotation(text="Crisis", x=2001, y=160, showarrow=False)
    fig.add_annotation(text="Kircherismo I", x=2005, y=160, showarrow=False)
    fig.add_annotation(text="Kircherismo II", x=2009, y=185, showarrow=False)
    fig.add_annotation(text="Kircherismo III", x=2013, y=160, showarrow=False)
    fig.add_annotation(text="Macrismo", x=2017, y=165, showarrow=False)
    fig.add_annotation(text="Alberto F.", x=2021, y=135, showarrow=False)

    #Nota al pie
    note = 'Fuente: INDEC'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.25,yref='paper', y=-0.15, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return fig

plot_serie_indices_precios_anuales()

In [ ]:
plot_serie_indices_anuales().write_html("../output/serie historica.html")

In [ ]:
#Importar datos descargados 
columnas_rubro=['anio','ngexpo_v','ngexpo_p','ngexpo_q','d1',
'pp_v','pp_p','pp_q','d2',
'moa_v','moa_p','moa_q','d3',
'moi_v','moi_p','moi_q','d4',
'cye_v','cye_p','cye_q']

columnas_uso=['anio','ngimpo_v','ngimpo_p','ngimpo_q','d1',
'bk_v','bk_p','bk_q','d2',
'bi_v','bi_p','bi_q','d3',
'cyl_v','cyl_p','cyl_q','d4',
'pya_v','pya_p','pya_q','d5',
'bc_v','bc_p','bc_q','d6',
'veh_v','veh_p','veh_q']

df_anual_rubro_indices=pd.read_excel('../downloads/indicesexpgr_04.xls', names=columnas_rubro, header=4, skipfooter=6)
df_anual_rubro_indices = df_anual_rubro_indices[df_anual_rubro_indices.columns.drop(list(df_anual_rubro_indices.filter(regex='d')))]
df_anual_rubro_indices.anio=df_anual_rubro_indices.anio.apply(lambda x: x.replace('*',''))

df_anual_uso_indices=pd.read_excel('../downloads/indicesimpue_04.xls',names=columnas_uso,header=4,skipfooter=6)
df_anual_uso_indices = df_anual_uso_indices[df_anual_uso_indices.columns.drop(list(df_anual_uso_indices.filter(regex='d')))]
df_anual_uso_indices.anio=df_anual_uso_indices.anio.apply(lambda x: x.replace('*',''))

columnas_rubro.insert(1,'mes')
df_mensual_rubro_indices=pd.read_excel('../downloads/serie_mensual_indices_expo.xls', names=columnas_rubro, header=4, skipfooter=6)
df_mensual_rubro_indices = df_mensual_rubro_indices[df_mensual_rubro_indices.columns.drop(list(df_mensual_rubro_indices.filter(regex='d')))]
df_mensual_rubro_indices.anio.fillna(method='ffill',inplace=True)
df_mensual_rubro_indices.anio=df_mensual_rubro_indices.anio.apply(lambda x: x.replace('*',''))

columnas_uso.insert(1,'mes')
df_mensual_uso_indices=pd.read_excel('../downloads/serie_mensual_indices_impo_ue.xls',names=columnas_uso,header=4,skipfooter=6)
df_mensual_uso_indices = df_mensual_uso_indices[df_mensual_uso_indices.columns.drop(list(df_mensual_uso_indices.filter(regex='d')))]
df_mensual_uso_indices.anio.fillna(method='ffill',inplace=True)
df_mensual_uso_indices.anio=df_mensual_uso_indices.anio.apply(lambda x: x.replace('*',''))

In [ ]:
df_anual_indices=pd.merge(left=df_anual_rubro_indices,right=df_anual_uso_indices, how='left', on='anio')
df_anual_indices['iti']=df_anual_indices.ngexpo_p/df_anual_indices.ngimpo_p*100
df_mensual_indices=pd.merge(left=df_mensual_rubro_indices,right=df_mensual_uso_indices,how='left',on=['anio','mes'])
df_mensual_indices['iti']=df_mensual_indices.ngexpo_p/df_mensual_indices.ngimpo_p*100

In [ ]:
dict_meses=dict(zip(df_mensual_indices.mes.unique(),range(1,13)))
def meses_num(mes):
    return dict_meses[mes]
df_mensual_indices['fecha']=df_mensual_indices.mes.apply(meses_num).astype(str)+'-'+df_mensual_indices['anio'].astype(str)
df_mensual_indices['fecha']=pd.to_datetime(df_mensual_indices["fecha"]).dt.strftime('%m-%Y')

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_3648\4039606152.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [ ]:
df_mensual_indices.to_csv('../output/indices_mensuales.csv')
df_anual_indices.to_csv("../output/indices_anuales.csv")

In [ ]:
df_mensual_rubro_indices['fecha']=df_mensual_rubro_indices.mes.apply(meses_num).astype(str)+'-'+df_mensual_rubro_indices['anio'].astype(str)
df_mensual_rubro_indices['fecha']=pd.to_datetime(df_mensual_rubro_indices["fecha"]).dt.strftime('%m-%Y')
df_mensual_rubro_indices.insert(2,'fecha',df_mensual_rubro_indices.pop('fecha'))
df_mensual_rubro_indices['var_q_pp']=df_mensual_rubro_indices.pp_q.pct_change(12)*100
df_mensual_rubro_indices['var_q_moa']=df_mensual_rubro_indices.moa_q.pct_change(12)*100
df_mensual_rubro_indices['var_p_pp']=df_mensual_rubro_indices.pp_p.pct_change(12)*100
df_mensual_rubro_indices['var_p_moa']=df_mensual_rubro_indices.moa_p.pct_change(12)*100
df_mensual_rubro_indices['var_p_moi']=df_mensual_rubro_indices.pp_p.pct_change(12)*100
df_mensual_rubro_indices['var_p_cye']=df_mensual_rubro_indices.moa_p.pct_change(12)*100
df_mensual_rubro_indices['var_q_moi']=df_mensual_rubro_indices.pp_q.pct_change(12)*100
df_mensual_rubro_indices['var_q_cye']=df_mensual_rubro_indices.moa_q.pct_change(12)*100
df_mensual_rubro_indices.columns

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_3648\2169843688.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Index(['anio', 'mes', 'fecha', 'ngexpo_v', 'ngexpo_p', 'ngexpo_q', 'pp_v',
       'pp_p', 'pp_q', 'moa_v', 'moa_p', 'moa_q', 'moi_v', 'moi_p', 'moi_q',
       'cye_v', 'cye_p', 'cye_q', 'var_q_pp', 'var_q_moa', 'var_p_pp',
       'var_p_moa', 'var_p_moi', 'var_p_cye', 'var_q_moi', 'var_q_cye'],
      dtype='object')

In [ ]:
df_mensual_rubro_indices_filtrado=df_mensual_rubro_indices[df_mensual_rubro_indices.anio>'2021']
x = df_mensual_rubro_indices_filtrado["fecha"]
#Datos
IIX_grafico = go.Figure()

IIX_grafico.add_trace(go.Scatter( name = "Variac cantid prod primarios", x=x, 
y=df_mensual_rubro_indices_filtrado["var_q_pp"], mode = "lines+markers", marker_symbol=0))
IIX_grafico.add_trace(go.Scatter( name = "Variac cantid manuf de origen agropecuario", x=x, 
y=df_mensual_rubro_indices_filtrado["var_q_moa"], mode = "lines+markers", marker_symbol=1))
IIX_grafico.add_trace(go.Scatter( name = "Variac precios prod primarios", x=x, 
y=df_mensual_rubro_indices_filtrado["var_p_pp"], mode = "lines+markers", marker_symbol=2))
IIX_grafico.add_trace(go.Scatter( name = "Variac precios manuf de origen agropecuario", x=x, 
y=df_mensual_rubro_indices_filtrado["var_p_moa"], mode = "lines+markers", marker_symbol=3))

#Propiedades
IIX_grafico.update_xaxes(showgrid=True)
IIX_grafico.update_yaxes(title_text="Variacion interanual índices. Base 2004 = 100")
IIX_grafico.update_layout(separators=",.", font_family="Georgia",  
# margin ={'t': 0},
height=600, width=1000,
# xaxis_range = [1989.5,2021.5],
xaxis = dict(tickmode = "linear"),
template = 'none',
title_text="Inicio de la pulseada política: exportaciones estancadas <br> <sup> Variación % interanual",
legend = dict(yanchor="top",y=0.99, xanchor="left",x=0.7)
)

#Nota al pie
note = 'Fuente: INDEC'
IIX_grafico.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.15,yref='paper', y=-0.18, 
                           xanchor='right', yanchor='auto', xshift=0, yshift=0,)

In [ ]:
multiplot_indices = make_subplots(2, 2, vertical_spacing=0.2,
subplot_titles= ['Productos primarios','Manufacturas de origen agropecuario', 'Manufacturas de origen industrial', 'Combustibles y energía'])
w=[1,1,2,2]
z=[1,2,1,2]
x=df_mensual_rubro_indices_filtrado.fecha

multiplot_indices.add_trace(go.Scatter(name='Variación Cantidades',legendgroup="Variación Cantidades",x=x, y=df_mensual_rubro_indices_filtrado.var_q_pp, mode='lines+markers', marker_symbol=0, marker_color='orange'), row=1, col=1)
multiplot_indices.add_trace(go.Scatter(name='Variación Precio',legendgroup="Variación Precio",x=x, y=df_mensual_rubro_indices_filtrado.var_p_pp, mode='lines+markers', marker_symbol=1, marker_color='purple'), row=1, col=1)

multiplot_indices.add_trace(go.Scatter(name='Variación Cantidades moa',legendgroup="Variación Cantidades",x=x, y=df_mensual_rubro_indices_filtrado.var_q_moa, mode='lines+markers', marker_symbol=0,marker_color='orange',showlegend=False), row=1, col=2)
multiplot_indices.add_trace(go.Scatter(name='Variación Precio moa',legendgroup="Variación Precio",x=x, y=df_mensual_rubro_indices_filtrado.var_p_moa, mode='lines+markers', marker_symbol=1, marker_color='purple',showlegend=False), row=1, col=2)

multiplot_indices.add_trace(go.Scatter(name='Variación Cantidades moi',legendgroup="Variación Cantidades",x=x, y=df_mensual_rubro_indices_filtrado.var_q_moi, mode='lines+markers', marker_symbol=0,marker_color='orange',showlegend=False ), row=2, col=1)
multiplot_indices.add_trace(go.Scatter(name='Variación Precio moi',legendgroup="Variación Precio",x=x, y=df_mensual_rubro_indices_filtrado.var_p_moi, mode='lines+markers', marker_symbol=1, marker_color='purple',showlegend=False), row=2, col=1)

multiplot_indices.add_trace(go.Scatter(name='Variación Cantidades cye',legendgroup="Variación Cantidades",x=x, y=df_mensual_rubro_indices_filtrado.var_q_cye, mode='lines+markers', marker_symbol=0,marker_color='orange',showlegend=False ), row=2, col=2)
multiplot_indices.add_trace(go.Scatter(name='Variación Precio cye',legendgroup="Variación Precio",x=x, y=df_mensual_rubro_indices_filtrado.var_p_cye, mode='lines+markers', marker_symbol=1, marker_color='purple',showlegend=False), row=2, col=2)
   
multiplot_indices.update_layout(separators=",.", font_family="Georgia", font_size=12,
                              xaxis=dict(type='category'),
                                # margin ={'b': 200},
     height=700, width=1000, 
     # xaxis = dict(tickmode = "linear"),
     template = 'none',
     title=dict(
         text="Inicio de la pulseada política: exportaciones estancadas <br> <sup> Variación % interanual, índices 2004=100 </sup>",
         font_size=20,
           pad=dict(b=500)
        ),    
     legend=dict(yanchor="bottom",xanchor="left",
     y=-.15, x=0.3, 
     orientation='h')
     )
multiplot_indices.update_yaxes(tickformat=',')
note = 'Fuente: INDEC'
multiplot_indices.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.1,yref='paper', y=-0.125, 
                           xanchor='right', yanchor='auto', xshift=0, yshift=0,)

multiplot_indices.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
    xref='paper', yref='paper', x=0.5, y=0.5)

In [ ]:
multiplot_indices.write_html('../output/multiplots_indices.html')
plot_serie_indices_anuales().write_html("../output/indices anuales.html")

In [ ]:
for i in glob("../downloads/*", recursive = True): os.remove(i)